In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten,  Dense, Convolution2D
from keras.preprocessing.image import ImageDataGenerator
import os
import random
import shutil
from google.colab import drive

In [2]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
class Classifier:
  classifier=Sequential()
  input_shape = (256, 256, 3)
  image_shape = (256, 256)
  def addConvolution(self):
    self.classifier.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation="relu", input_shape=self.input_shape))

  def addMaxPooling(self):
    self.classifier.add(MaxPooling2D(pool_size=(2,2)))

  def addFlatten(self):
    self.classifier.add(Flatten())

  def addDense(self, units=128, activation="relu"):
    self.classifier.add(Dense(units=units, activation=activation))

  def compile(self, optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"]):
    self.classifier.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    self.classifier.summary()

In [4]:
model =  Classifier()

In [5]:
model.addConvolution()
model.addMaxPooling()
model.addConvolution()
model.addMaxPooling()
model.addFlatten()
model.addDense(activation="relu")
model.addDense(units=3, activation="sigmoid")
model.compile()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 128)               3

In [6]:
source = "/content/drive/MyDrive/dataset/training"
destination = "/content/drive/MyDrive/dataset/test"

In [66]:
def move_samples(source_folder, destination_folder):
    files = os.listdir(source_folder)
    random.shuffle(files)
    for file in files[:10]:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        print(f"moving {source_path} to {destination_path}")
        shutil.move(source_path, destination_path)

In [69]:
for folder in os.listdir(source):
    source_folder = os.path.join(source, folder)
    destination_folder = os.path.join(destination, folder)
    print(source_folder)
    print(destination_folder)
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    move_samples(source_folder, destination_folder)

/content/drive/MyDrive/dataset/training/iris-setosa
/content/drive/MyDrive/dataset/test/iris-setosa
moving /content/drive/MyDrive/dataset/training/iris-setosa/iris-fdd46bc8cdd760707811fe958f12f7a3aa7ccc51609d9ec710c074af0bf9fb42.jpg to /content/drive/MyDrive/dataset/test/iris-setosa/iris-fdd46bc8cdd760707811fe958f12f7a3aa7ccc51609d9ec710c074af0bf9fb42.jpg
moving /content/drive/MyDrive/dataset/training/iris-setosa/iris-0ff5ba898a0ec179a25ca217af45374fdd06d606bb85fc29294291facad1776a.jpg to /content/drive/MyDrive/dataset/test/iris-setosa/iris-0ff5ba898a0ec179a25ca217af45374fdd06d606bb85fc29294291facad1776a.jpg
moving /content/drive/MyDrive/dataset/training/iris-setosa/iris-f31c83878b37242ce9306e3bc59c557786e8d496841cae4a91638efd47d66ad8.jpg to /content/drive/MyDrive/dataset/test/iris-setosa/iris-f31c83878b37242ce9306e3bc59c557786e8d496841cae4a91638efd47d66ad8.jpg
moving /content/drive/MyDrive/dataset/training/iris-setosa/iris-f340f1f09cf8a0f83b28eb8e945317bb2e11502575275bb27ee8163ee28d27

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
from tensorflow.python.ops.batch_ops import batch
train_data = train_datagen.flow_from_directory(directory=source,
                                               target_size=model.image_shape,
                                               class_mode="categorical",
                                               batch_size=327*5
                                               )
test_data = test_datagen.flow_from_directory(directory=destination,
                                               target_size=model.image_shape,
                                               class_mode="categorical",
                                               batch_size=327*5
                                               )

Found 327 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


In [ ]:
model.classifier.fit(train_data, epochs=5, steps_per_epoch=327, validation_data=test_data)

Epoch 1/5
